In [12]:
import torch
import os
import random
import cv2
import numpy as np
import shutil

from collections import defaultdict
from PIL import Image
from torchvision.datasets import CIFAR100
from glob import glob
from tqdm import tqdm


In [13]:

# =====================
# Configurações Gerais
# =====================
RANDOM_SEED = 42
TARGET_SIZE_CIFAR = (256, 256)
TRAIN_PER_CLASS = 400
VAL_PER_CLASS = 80

# CIFAR-100
BASE_INPUT_DIR = "data"
BASE_OUTPUT_DIR = "datasets/cifar100"
TRAIN_DIR_CIFAR = os.path.join(BASE_OUTPUT_DIR, "train")
VAL_DIR_CIFAR = os.path.join(BASE_OUTPUT_DIR, "val")

#https://www.kaggle.com/datasets/mikhailma/test-dataset

# Google reCAPTCHA v2
RECAPTCHA_SOURCE_DIR = 'Google_Recaptchav2/images'
RECAPTCHA_TRAIN_DIR = 'datasets/recaptcha/train'
RECAPTCHA_VAL_DIR = 'datasets/recaptcha/val'

# =================================
# CIFAR-100: Salvamento de imagens
# =================================
def process_and_save_split(indices, dataset, save_dir, target_size):
    os.makedirs(save_dir, exist_ok=True)
    for idx in indices:
        img, label = dataset[idx]
        class_name = dataset.classes[label]
        img_resized = img.resize(target_size, Image.BILINEAR)
        class_dir = os.path.join(save_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
        img_filename = f"{idx:05d}.png"
        img_path = os.path.join(class_dir, img_filename)
        img_resized.save(img_path)

# =======================================
# Google reCAPTCHA v2: Cópia e divisão
# =======================================

def process_recaptcha_dataset():
    if not os.path.exists(RECAPTCHA_SOURCE_DIR):
        print(f"❌ O diretório '{RECAPTCHA_SOURCE_DIR}' não foi encontrado.")
        return
    else:
        print(f"📁 Diretório encontrado: '{RECAPTCHA_SOURCE_DIR}'")

    os.makedirs(RECAPTCHA_TRAIN_DIR, exist_ok=True)
    os.makedirs(RECAPTCHA_VAL_DIR, exist_ok=True)

    print("🔍 Lendo classes do reCAPTCHA v2...")
    classes = os.listdir(dataset_dir)

    for class_name in classes:
        class_path = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_path):
            os.makedirs(os.path.join(RECAPTCHA_TRAIN_DIR, class_name), exist_ok=True)
            os.makedirs(os.path.join(RECAPTCHA_VAL_DIR, class_name), exist_ok=True)

            images = [img for img in os.listdir(class_path) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
            random.shuffle(images)

            val_size = int(len(images) * 0.2)
            val_images = images[:val_size]
            train_images = images[val_size:]

            for idx, image in enumerate(train_images, 1):
                new_name = f"{idx}_{image}"
                shutil.copy(os.path.join(class_path, image), os.path.join(RECAPTCHA_TRAIN_DIR, class_name, new_name))

            for idx, image in enumerate(val_images, 1):
                new_name = f"{idx}_{image}"
                shutil.copy(os.path.join(class_path, image), os.path.join(RECAPTCHA_VAL_DIR, class_name, new_name))

    print("✅ reCAPTCHA v2: divisão 80/20 concluída!")
    
# ===============
# Pipeline geral
# ===============
def main():
    print("🔍 Carregando CIFAR-100 (train split)...")
    dataset = CIFAR100(root=BASE_INPUT_DIR, train=True, download=True)

    print("📊 Agrupando por classe...")
    class_to_indices = defaultdict(list)
    for idx, label in enumerate(dataset.targets):
        class_to_indices[label].append(idx)

    print("🔀 Embaralhando e separando...")
    random.seed(RANDOM_SEED)
    train_indices, val_indices = [], []
    for label, idx_list in class_to_indices.items():
        random.shuffle(idx_list)
        train_indices.extend(idx_list[:TRAIN_PER_CLASS])
        val_indices.extend(idx_list[TRAIN_PER_CLASS:TRAIN_PER_CLASS + VAL_PER_CLASS])

    print("💾 Salvando treino CIFAR-100...")
    process_and_save_split(train_indices, dataset, TRAIN_DIR_CIFAR, TARGET_SIZE_CIFAR)
    print(f"✅ {len(train_indices)} imagens de treino salvas.")

    print("💾 Salvando validação CIFAR-100...")
    process_and_save_split(val_indices, dataset, VAL_DIR_CIFAR, TARGET_SIZE_CIFAR)
    print(f"✅ {len(val_indices)} imagens de validação salvas.")

    print("🧠 Processando reCAPTCHA v2...")
    process_recaptcha_dataset()

    print("📄 Salvando log...")
    os.makedirs("outputs", exist_ok=True)
    with open("outputs/logs.txt", "w") as f:
        f.write(f"CIFAR-100 Treino: {len(train_indices)}\n")
        f.write(f"CIFAR-100 Validação: {len(val_indices)}\n")
        f.write(f"Resolução CIFAR: {TARGET_SIZE_CIFAR[0]}x{TARGET_SIZE_CIFAR[1]}\n")

    print("🏁 Tudo finalizado com sucesso!")

# ===============
# Execução
# ===============
if __name__ == "__main__":
    main()


🔍 Carregando CIFAR-100 (train split)...
📊 Agrupando por classe...
🔀 Embaralhando e separando...
💾 Salvando treino CIFAR-100...


KeyboardInterrupt: 

# Criação GRv2P (Google Recaptcha v2 pós-holdout) para treinamento dos Modelos

In [1]:

# Caminhos de origem e destino
dataset_dir = 'datasets/Google_Recaptchav2/images'
train_dir = 'datasets/GRv2P/train'
val_dir = 'datasets/GRv2P/val'

# Verificar se o diretório de origem existe
if not os.path.exists(dataset_dir):
    print(f"O diretório de origem '{dataset_dir}' não existe.")
else:
    print(f"O diretório de origem '{dataset_dir}' foi encontrado.")

# Criação das pastas de treino e validação, se não existirem
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Verificar se as pastas de destino foram criadas corretamente
print(f"Diretórios de treino: {train_dir}")
print(f"Diretórios de validação: {val_dir}")

# Listar as classes
classes = os.listdir(dataset_dir)

for class_name in classes:
    class_path = os.path.join(dataset_dir, class_name)

    # Verificar se é uma pasta e contornar se for
    if os.path.isdir(class_path):
        # Criar diretórios de classe dentro de train e val
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

        # Listar todas as imagens na classe
        images = os.listdir(class_path)
        images = [img for img in images if img.endswith(('.jpg', '.jpeg', '.png'))]  # Ajuste os formatos se necessário

        # Embaralhar as imagens aleatoriamente
        random.shuffle(images)

        # Determinar o tamanho para validação (15%)
        val_size = int(len(images) * 0.15)

        # Separar imagens para val e train
        val_images = images[:val_size]
        train_images = images[val_size:]

        # Copiar as imagens para a pasta correta (train e val) com número crescente no nome
        for idx, image in enumerate(train_images, 1):
            # Criar novo nome com número crescente
            new_image_name = f"{idx}_{image}"
            shutil.copy(os.path.join(class_path, image), os.path.join(train_dir, class_name, new_image_name))

        for idx, image in enumerate(val_images, 1):
            # Criar novo nome com número crescente
            new_image_name = f"{idx}_{image}"
            shutil.copy(os.path.join(class_path, image), os.path.join(val_dir, class_name, new_image_name))

print("Distribuição 80-20 concluída com cópias das imagens e números crescentes!")


O diretório de origem 'datasets/Google_Recaptchav2/images' foi encontrado.
Diretórios de treino: GRv2P/train
Diretórios de validação: GRv2P/val
Distribuição 80-20 concluída com cópias das imagens e números crescentes!


# Criação/cópia do dataset GRv2P após aplicar ruido Gaussiano

In [11]:
# Caminhos
val_dir = 'datasets/GRv2P/val'
val_aug_dir = 'datasets/GRv2P/val_aug'

# Cria diretório de saída base
os.makedirs(val_aug_dir, exist_ok=True)

def add_gaussian_noise(image, mean=0, std=15):
    noise = np.random.normal(mean, std, image.shape).astype(np.float32)
    noisy = image.astype(np.float32) + noise
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

def wave_distortion(image, amplitude=1, wavelength=1.6):
    rows, cols = image.shape[:2]
    map_x = np.zeros((rows, cols), dtype=np.float32)
    map_y = np.zeros((rows, cols), dtype=np.float32)

    for i in range(rows):
        for j in range(cols):
            dx = amplitude * np.sin(2 * np.pi * i / wavelength)
            dy = amplitude * np.cos(2 * np.pi * j / wavelength)
            map_x[i, j] = j + dx
            map_y[i, j] = i + dy

    return cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

# Percorre todas as imagens nas subpastas
image_paths = glob(os.path.join(val_dir, '**', '*.*'), recursive=True)

for img_path in tqdm(image_paths, desc="Aplicando ruído e distorção"):
    img = cv2.imread(img_path)
    if img is None:
        continue

    # Aplica filtros
    noisy_img = add_gaussian_noise(img)
    distorted_img = wave_distortion(noisy_img)

    # Caminho relativo da imagem para manter a estrutura de pastas
    rel_path = os.path.relpath(img_path, val_dir)
    save_path = os.path.join(val_aug_dir, rel_path)

    # Cria subdiretório se necessário
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Salva imagem
    cv2.imwrite(save_path, distorted_img)


Aplicando ruído e distorção: 100%|█████████████████████████████████████████████████| 1754/1754 [01:04<00:00, 27.29it/s]


# Criação/cópia do dataset CIFAR-100 após aplicar ruido Gaussiano

In [14]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Diretório onde estão as imagens com ruído e distorção
val_dir = 'datasets/cifar100/val'       # imagens originais
val_aug_dir = 'datasets/cifar100/val_aug'  # onde vamos salvar as distorcidas

# Cria diretório de saída base
os.makedirs(val_aug_dir, exist_ok=True)

# Filtros
def add_gaussian_noise(image, mean=0, std=15):
    noise = np.random.normal(mean, std, image.shape).astype(np.float32)
    noisy = image.astype(np.float32) + noise
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

def wave_distortion(image, amplitude=1, wavelength=1.6):
    rows, cols = image.shape[:2]
    map_x = np.zeros((rows, cols), dtype=np.float32)
    map_y = np.zeros((rows, cols), dtype=np.float32)

    for i in range(rows):
        for j in range(cols):
            dx = amplitude * np.sin(2 * np.pi * i / wavelength)
            dy = amplitude * np.cos(2 * np.pi * j / wavelength)
            map_x[i, j] = j + dx
            map_y[i, j] = i + dy

    return cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

# Aplica ruído e distorção
image_paths = glob(os.path.join(val_dir, '**', '*.*'), recursive=True)
for img_path in tqdm(image_paths, desc="Aplicando ruído e distorção"):
    img = cv2.imread(img_path)
    if img is None:
        continue

    noisy_img = add_gaussian_noise(img)
    distorted_img = wave_distortion(noisy_img)

    # Caminho relativo e novo nome com "aug"
    rel_path = os.path.relpath(img_path, val_dir)
    base, ext = os.path.splitext(rel_path)
    new_name = base + "_aug" + ext
    save_path = os.path.join(val_aug_dir, new_name)

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    cv2.imwrite(save_path, distorted_img)


Aplicando ruído e distorção: 100%|█████████████████████████████████████████████████| 8000/8000 [19:18<00:00,  6.90it/s]
